## Restore Databases
Restore the AdventureWorks database three times: All from the same backup file, but with three different names. You'll need to download the Backup file from [Github](https://docs.microsoft.com/en-us/sql/samples/adventureworks-install-configure?view=sql-server-2017).

*You will also have to adjust paths for the file locations: both of the backup & the restore location*

In [1]:
USE master
GO

RESTORE DATABASE AW_NONE
FROM DISK = 'C:\SQL\BAK\AdventureWorks2014.bak'
WITH MOVE 'AdventureWorks2014_Data' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_NONE_Data.mdf'
    ,MOVE 'AdventureWorks2014_Log' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_NONE_Log.ldf'
    ,REPLACE;

RESTORE DATABASE AW_ROW
FROM DISK = 'C:\SQL\BAK\AdventureWorks2014.bak'
WITH MOVE 'AdventureWorks2014_Data' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_ROW_Data.mdf'
    ,MOVE 'AdventureWorks2014_Log' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_ROW_Log.ldf'
    ,REPLACE;


RESTORE DATABASE AW_PAGE
FROM DISK = 'C:\SQL\BAK\AdventureWorks2014.bak'
WITH MOVE 'AdventureWorks2014_Data' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_PAGE_Data.mdf'
    ,MOVE 'AdventureWorks2014_Log' TO 'C:\SQL\MSSQL15.MSSQLSERVER\MSSQL\DATA\AW_PAGE_Log.ldf'
    ,REPLACE;




## Create `dbo.ObjectSize` as a view to easily look at how large objects are
We'll do this in all three databases.

I'm using the `CREATE OR ALTER` syntax that is supported in SQL Server 2016+.

In [2]:
USE AW_NONE
GO
CREATE OR ALTER VIEW dbo.ObjectSize
AS
SELECT DatabaseName    = 'AW_NONE',
       ObjectName      = o.name, 
       IndexName       = i.name,
       PageType        = au.type_desc,
       CompressionType = p.data_compression_desc, 
       NumberRows      = p.rows, 
       TotalPages      = au.total_pages, 
       TotalSizeKB     = au.total_pages*8
FROM sys.objects o
JOIN sys.indexes i ON i.object_id = o.object_id
JOIN sys.partitions p ON o.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.allocation_units au ON p.hobt_id = au.container_id;
GO
--
--
USE AW_ROW
GO
CREATE OR ALTER VIEW dbo.ObjectSize
AS
SELECT DatabaseName    = 'AW_ROW',
       ObjectName      = o.name, 
       IndexName       = i.name,
       PageType        = au.type_desc,
       CompressionType = p.data_compression_desc, 
       NumberRows      = p.rows, 
       TotalPages      = au.total_pages, 
       TotalSizeKB     = au.total_pages*8
FROM sys.objects o
JOIN sys.indexes i ON i.object_id = o.object_id
JOIN sys.partitions p ON o.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.allocation_units au ON p.hobt_id = au.container_id;
GO
--
--
USE AW_PAGE
GO
CREATE OR ALTER VIEW dbo.ObjectSize
AS
SELECT DatabaseName    = 'AW_PAGE',
       ObjectName      = o.name, 
       IndexName       = i.name,
       PageType        = au.type_desc,
       CompressionType = p.data_compression_desc, 
       NumberRows      = p.rows, 
       TotalPages      = au.total_pages, 
       TotalSizeKB     = au.total_pages*8
FROM sys.objects o
JOIN sys.indexes i ON i.object_id = o.object_id
JOIN sys.partitions p ON o.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.allocation_units au ON p.hobt_id = au.container_id;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.062

## Lets use the view to show all three databases are still the same

In [3]:
SELECT DatabaseName,   
       NumberRows      = SUM(NumberRows), 
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_NONE.dbo.ObjectSize
GROUP BY DatabaseName
UNION ALL
SELECT DatabaseName,   
       NumberRows      = SUM(NumberRows), 
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_ROW.dbo.ObjectSize
GROUP BY DatabaseName
UNION ALL
SELECT DatabaseName,   
       NumberRows      = SUM(NumberRows), 
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_PAGE.dbo.ObjectSize
GROUP BY DatabaseName
ORDER BY TotalSizeKB DESC;

(3 rows affected)

Total execution time: 00:00:00.944

DatabaseName,NumberRows,TotalPages,TotalSizeKB
AW_NONE,4959992,24465,195720
AW_PAGE,4959992,24465,195720
AW_ROW,4959992,24465,195720


See? Proof they are all exactly the same size (so far).

## Rebuild indexes in all three databases
Each database will get indexes rebuilt using a different compression level. 
Create this `REBUILD` proc in each of the databases, then execute it.

In [4]:
/*
     ███╗   ██╗ ██████╗ ███╗   ██╗███████╗    
     ████╗  ██║██╔═══██╗████╗  ██║██╔════╝    
     ██╔██╗ ██║██║   ██║██╔██╗ ██║█████╗      
     ██║╚██╗██║██║   ██║██║╚██╗██║██╔══╝      
     ██║ ╚████║╚██████╔╝██║ ╚████║███████╗    
     ╚═╝  ╚═══╝ ╚═════╝ ╚═╝  ╚═══╝╚══════╝   
*/
USE AW_NONE
GO
CREATE OR ALTER PROCEDURE dbo.Rebuild_All_Indexes @CompressionLevel nvarchar(4)
    AS
DECLARE @sql nvarchar(max) = '';
--
SELECT @sql += N'ALTER TABLE [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type = 0  --Just heaps
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
SELECT @sql += N'ALTER INDEX [' + i.name + N'] ON [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type NOT IN (0,3) --Exclude heaps & XML indexes
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
PRINT 'Rebuilding indexes with ' + @CompressionLevel + ' compression...'
EXEC sp_executesql @sql;
GO
/*
     ██████╗  ██████╗ ██╗    ██╗
     ██╔══██╗██╔═══██╗██║    ██║
     ██████╔╝██║   ██║██║ █╗ ██║
     ██╔══██╗██║   ██║██║███╗██║
     ██║  ██║╚██████╔╝╚███╔███╔╝
     ╚═╝  ╚═╝ ╚═════╝  ╚══╝╚══╝ 
*/
USE AW_ROW
GO

CREATE OR ALTER PROCEDURE dbo.Rebuild_All_Indexes @CompressionLevel nvarchar(4)
    AS
DECLARE @sql nvarchar(max) = '';
--
SELECT @sql += N'ALTER TABLE [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type = 0  --Just heaps
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
SELECT @sql += N'ALTER INDEX [' + i.name + N'] ON [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type NOT IN (0,3) --Exclude heaps & XML indexes
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
PRINT 'Rebuilding indexes with ' + @CompressionLevel + ' compression...'
EXEC sp_executesql @sql;
GO
/*
     ██████╗  █████╗  ██████╗ ███████╗
     ██╔══██╗██╔══██╗██╔════╝ ██╔════╝
     ██████╔╝███████║██║  ███╗█████╗  
     ██╔═══╝ ██╔══██║██║   ██║██╔══╝  
     ██║     ██║  ██║╚██████╔╝███████╗
     ╚═╝     ╚═╝  ╚═╝ ╚═════╝ ╚══════╝
*/
USE AW_PAGE
GO

CREATE OR ALTER PROCEDURE dbo.Rebuild_All_Indexes @CompressionLevel nvarchar(4)
    AS
DECLARE @sql nvarchar(max) = '';
--
SELECT @sql += N'ALTER TABLE [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type = 0  --Just heaps
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
SELECT @sql += N'ALTER INDEX [' + i.name + N'] ON [' + s.name + N'].[' + t.name + N'] REBUILD WITH (DATA_COMPRESSION=' + @CompressionLevel + N');'
FROM sys.objects t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.indexes i ON i.object_id = t.object_id
WHERE i.type NOT IN (0,3) --Exclude heaps & XML indexes
AND t.type IN ('U','V')
ORDER BY s.name, t.name;
--
PRINT 'Rebuilding indexes with ' + @CompressionLevel + ' compression...'
EXEC sp_executesql @sql;
GO
/*
███████╗██╗  ██╗███████╗ ██████╗██╗   ██╗████████╗███████╗
██╔════╝╚██╗██╔╝██╔════╝██╔════╝██║   ██║╚══██╔══╝██╔════╝
█████╗   ╚███╔╝ █████╗  ██║     ██║   ██║   ██║   █████╗  
██╔══╝   ██╔██╗ ██╔══╝  ██║     ██║   ██║   ██║   ██╔══╝  
███████╗██╔╝ ██╗███████╗╚██████╗╚██████╔╝   ██║   ███████╗
╚══════╝╚═╝  ╚═╝╚══════╝ ╚═════╝ ╚═════╝    ╚═╝   ╚══════╝
*/
EXEC AW_NONE.dbo.Rebuild_All_Indexes 'NONE';
GO
EXEC  AW_ROW.dbo.Rebuild_All_Indexes 'ROW';
GO
EXEC AW_PAGE.dbo.Rebuild_All_Indexes 'PAGE';
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Rebuilding indexes with NONE compression...

Rebuilding indexes with ROW compression...

Rebuilding indexes with PAGE compression...

Total execution time: 00:00:24.953

## OK, now let's check the size again

In [38]:
SELECT DatabaseName,   
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_NONE.dbo.ObjectSize
GROUP BY DatabaseName
UNION ALL
SELECT DatabaseName,   
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_ROW.dbo.ObjectSize
GROUP BY DatabaseName
UNION ALL
SELECT DatabaseName,   
       TotalPages      = SUM(TotalPages), 
       TotalSizeKB     = SUM(TotalSizeKB)
FROM AW_PAGE.dbo.ObjectSize
GROUP BY DatabaseName
ORDER BY TotalSizeKB DESC;

## Let's look at some specific tables!

In [15]:
SELECT DatabaseName, ObjectName, TotalSizeKB = SUM(TotalSizeKB)
FROM AW_NONE.dbo.ObjectSize      
WHERE ObjectName IN ('SalesOrderDetail','Person','JobCandidate','vProductAndDescription')
GROUP BY DatabaseName, ObjectName
UNION
SELECT DatabaseName, ObjectName, TotalSizeKB = SUM(TotalSizeKB)
FROM AW_ROW.dbo.ObjectSize      
WHERE ObjectName IN ('SalesOrderDetail','Person','JobCandidate','vProductAndDescription')
GROUP BY DatabaseName, ObjectName
UNION
SELECT DatabaseName, ObjectName, TotalSizeKB = SUM(TotalSizeKB)
FROM AW_PAGE.dbo.ObjectSize      
WHERE ObjectName IN ('SalesOrderDetail','Person','JobCandidate','vProductAndDescription')
GROUP BY DatabaseName, ObjectName
ORDER BY ObjectName, TotalSizeKb DESC

## Results

* Job Candidate table is the same size, regardless of compression
* Person saves 2% with ROW compression, and an *additional* 4% with PAGE compression
* SalesOrderDetail saves 19% with ROW compression and an *additional* 18% with PAGE compression
* vProductAndDescription saves 40% with ROW compression but only an additional 64kb with PAGE compression

## Lets look at `HumanResources.JobCandidate`

* Job Candidate table is the same size, regardless of compression.



In [16]:
USE AW_PAGE

SELECT DbName             = db_name(database_id) 
      ,TableName          = object_name(object_id,database_id) 
      ,IndexID            = index_id
      ,IndexType          = index_type_desc
      ,AllocUnitType      = alloc_unit_type_desc
      ,IndexSizeKB        = page_count * 8
      ,CompressedPages    = compressed_page_count
      ,UncompressedPages  = page_count - compressed_page_count
      ,AvgRecordSizeBytes = avg_record_size_in_bytes
FROM sys.dm_db_index_physical_stats(db_id('AW_PAGE'),
                                    object_id('HumanResources.JobCandidate'),
                                    NULL,NULL,'Detailed') -- Have to use Detailed mode to get compression info
WHERE index_level = 0 --Only the leaf level of the index
ORDER BY compressed_page_count DESC, 
         page_count DESC;

## Let's look at the data in the table

In [17]:
SELECT * FROM HumanResources.JobCandidate;

## Lets look at `Production.vProductAndDescription`

* vProductAndDescription saves 40% with ROW compression but only an additional 64kb with PAGE compression

In [18]:
USE AW_PAGE

SELECT DbName             = db_name(database_id) 
      ,TableName          = object_name(object_id,database_id) 
      ,IndexID            = index_id
      ,IndexType          = index_type_desc
      ,AllocUnitType      = alloc_unit_type_desc
      ,IndexSizeKB        = page_count * 8
      ,CompressedPages    = compressed_page_count
      ,UncompressedPages  = page_count - compressed_page_count
      ,AvgRecordSizeBytes = avg_record_size_in_bytes
FROM sys.dm_db_index_physical_stats(db_id('AW_PAGE'),
                                    object_id('Production.vProductAndDescription'),
                                    NULL,NULL,'Detailed') -- Have to use Detailed mode to get compression info
WHERE index_level = 0 --Only the leaf level of the index
ORDER BY compressed_page_count DESC, 
         page_count DESC;

## Let's look at the data in the table

In [20]:
SELECT TOP 100 * FROM Production.vProductAndDescription;

## Lets look at `Person.Person`

* Person saves 2% with ROW compression, and an *additional* 4% with PAGE compression

In [21]:
USE AW_PAGE

SELECT DbName             = db_name(database_id) 
      ,TableName          = object_name(object_id,database_id) 
      ,IndexID            = index_id
      ,IndexType          = index_type_desc
      ,AllocUnitType      = alloc_unit_type_desc
      ,IndexSizeKB        = page_count * 8
      ,CompressedPages    = compressed_page_count
      ,UncompressedPages  = page_count - compressed_page_count
      ,AvgRecordSizeBytes = avg_record_size_in_bytes
FROM sys.dm_db_index_physical_stats(db_id('AW_PAGE'),
                                    object_id('Person.Person'),
                                    NULL,NULL,'Detailed') -- Have to use Detailed mode to get compression info
WHERE index_level = 0 --Only the leaf level of the index
AND index_id < 256000  --Exclude XML indexes
ORDER BY compressed_page_count DESC, 
         page_count DESC;

## What's  Index_Id 3 and why isn't it compressed?

In [22]:
USE AW_PAGE;
EXEC sp_SQLskills_helpindex 'Person.Person';

## Let's look at the data in the table

In [23]:
SELECT TOP 50 rowguid
FROM Person.Person
ORDER BY rowguid;

## Lets look at `Sales.SalesOrderDetail`

* SalesOrderDetail saves 19% with ROW compression and an *additional* 18% with PAGE compression

Let's look at the indexes.... Which ones might be good candidates for compression?

In [24]:
USE AW_PAGE
EXEC sp_SQLskills_helpindex 'Sales.SalesOrderDetail'

## Let's look at the data in the table
### `PK_SalesOrderDetail_SalesOrderID_SalesOrderDetailID`

In [25]:
USE AW_PAGE

SELECT TOP 50 SalesOrderID, SalesOrderDetailID
FROM Sales.SalesOrderDetail
ORDER BY SalesOrderID, SalesOrderDetailID;

### `AK_SalesOrderDetail_rowguid`

In [26]:
USE AW_PAGE

SELECT TOP 50 rowguid, SalesOrderID, SalesOrderDetailID
FROM Sales.SalesOrderDetail
ORDER BY rowguid;

### `IX_SalesOrderDetail_ProductID`

In [27]:
USE AW_PAGE

SELECT TOP 50 ProductID, SalesOrderID, SalesOrderDetailID
FROM Sales.SalesOrderDetail
ORDER BY ProductID;

## One more thing... Maintenance Cost

In [32]:
EXEC AW_NONE.dbo.Rebuild_All_Indexes 'NONE';
GO
EXEC  AW_ROW.dbo.Rebuild_All_Indexes 'ROW';
GO
EXEC AW_PAGE.dbo.Rebuild_All_Indexes 'PAGE';
GO